## Create Vector DB

In [1]:
import polars as pl
import pickle

from LegalDefAgent.src.utils import setup_logging
from LegalDefAgent.src.retriever.extractor import DefinitionExtractor
from LegalDefAgent.src.retriever.embedder import VectorDBBuilder

setup_logging()

2024-12-09 16:28:05,069 - INFO - Logging configured


### Extract Definitions

#### Create df

In [6]:
extractor = DefinitionExtractor()
processed_df = extractor.extract_and_filter()

2024-12-09 15:23:12,679 - INFO - Extracting definitions from XML files in dataset EurLex...
2024-12-09 15:23:32,958 - INFO - Extracting definitions from XML files in dataset Normattiva...
2024-12-09 15:23:47,275 - INFO - Extracting definitions from XML files in dataset PDL...
2024-12-09 15:23:48,314 - INFO - Extracted definitions from 1348 files. 0 errors out of 22187 files.


In [10]:
processed_df.head()

id,definition_text,def_n,dataset,document_id,references
u32,str,str,str,str,list[str]
0,"""obstacles to trade: means any …","""#def_1""","""EurLex""","""32015R1843.xml""",[]
1,"""injury: means any material inj…","""#def_2""","""EurLex""","""32015R1843.xml""",[]
2,"""adverse trade effects: means t…","""#def_3""","""EurLex""","""32015R1843.xml""",[]
3,"""Union industry: means either: …","""#def_4""","""EurLex""","""32015R1843.xml""",[]
4,"""Union enterprise: means a comp…","""#def_5""","""EurLex""","""32015R1843.xml""",[]


#### Save df to parquet

In [25]:
processed_df.write_parquet("../data/processed_definitions.parquet")

### Create Embeddings

#### Load df if created previously

In [3]:
processed_df = pl.read_parquet("../data/processed_definitions.parquet")

### Insert into DB

### Load embeddings

In [4]:
with open('../data/def_embeddings.pkl', 'rb') as f:
    defs_embeddings = pickle.load(f)

In [40]:
from pathlib import Path
u = '/home/leo/Desktop/dhdk/Master thesis/.project/LegalDefAgent/vec_db/definitions_vectors.db'
Path(u).parent

PosixPath('/home/leo/Desktop/dhdk/Master thesis/.project/LegalDefAgent/vec_db')

In [10]:
db_builder = VectorDBBuilder()
db_builder.build_vector_db(processed_df, defs_embeddings)

2024-12-09 16:30:13,151 - DEBUG - https://huggingface.co:443 "GET /api/models/BAAI/bge-m3/revision/main HTTP/1.1" 200 4839


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

2024-12-09 16:30:14,588 - INFO - loading existing colbert_linear and sparse_linear---------
2024-12-09 16:30:14,662 - INFO - Building vector database...
2024-12-09 16:30:14,663 - INFO - Pass in the local path /home/leo/Desktop/dhdk/Master thesis/.project/LegalDefAgent/vec_db/definitions_vectors.db, and run it using milvus-lite
2024-12-09 16:30:14,668 - DEBUG - Created new connection using: 3dce3167eb17457a9e6b6671065c2d42
2024-12-09 16:30:14,669 - INFO - Pass in the local path /home/leo/Desktop/dhdk/Master thesis/.project/LegalDefAgent/vec_db/definitions_vectors.db, and run it using milvus-lite
2024-12-09 16:30:27,192 - INFO - Inserted 14729 entities into vector database
